In [1]:
import torch.nn as nn
import torch

In [2]:
class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.linear(x) + h)
        return new_h

In [3]:
my_cell = MyCell()

In [4]:
x, h = torch.rand(3, 4), torch.rand(3, 4)

In [5]:
traced_cell = torch.jit.trace(my_cell, (x, h))

In [6]:
traced_cell

TracedModule[MyCell](
  original_name=MyCell
  (linear): TracedModule[Linear](original_name=Linear)
)

In [7]:
traced_cell(x, h)

tensor([[ 0.4238, -0.0524,  0.5719,  0.4747],
        [-0.0059, -0.3625,  0.2658,  0.7130],
        [ 0.4532,  0.6390,  0.6385,  0.6584]],
       grad_fn=<DifferentiableGraphBackward>)